In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3

from utils.data_prep_utils import LABELS, DATES, prep_data
from utils.train_utils import prep_data_train_val_test, set_curr_time, show_sample

In [3]:
labels = ['zeev', 'or', 'felix', 'ron', 'aviya']

In [4]:
prep_data(DATES, labels, frames_size=128)

100%|██████████| 3/3 [00:01<00:00,  1.89it/s]
0it [00:00, ?it/s]
100%|██████████| 3/3 [00:01<00:00,  1.87it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 30/30 [00:16<00:00,  1.81it/s]
0it [00:00, ?it/s]
100%|██████████| 21/21 [00:11<00:00,  1.89it/s]
0it [00:00, ?it/s]
100%|██████████| 21/21 [00:11<00:00,  1.82it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [5]:
x_train, x_valid, x_test, y_train, y_valid, y_test = prep_data_train_val_test(DATES, labels, frames_size=128)

100%|██████████| 3000/3000 [00:01<00:00, 2002.86it/s]
0it [00:00, ?it/s]
100%|██████████| 3000/3000 [00:01<00:00, 1887.46it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 30000/30000 [00:15<00:00, 1969.88it/s]
0it [00:00, ?it/s]
100%|██████████| 21000/21000 [00:10<00:00, 1971.53it/s]
0it [00:00, ?it/s]
100%|██████████| 21000/21000 [00:10<00:00, 1991.26it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [6]:
x_train_rgb = np.repeat(x_train[..., np.newaxis], 3, -1)
x_val_rgb = np.repeat(x_valid[..., np.newaxis], 3, -1)
x_test_rgb = np.repeat(x_test[..., np.newaxis], 3, -1)

In [7]:
input_tensor = Input(shape=(128, 128, 3))
inceptionv3_model = InceptionV3(input_tensor=input_tensor, weights=None, include_top=True, classes=len(labels))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
for layer in inceptionv3_model.layers:
    layer.trainable = True

In [10]:
inceptionv3_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [11]:
curr_dt = set_curr_time()
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs/{0}_{1}'.format('inceptionv3', curr_dt),
                                             histogram_freq=0, write_graph=True)

In [13]:
inceptionv3_model.fit(x_train_rgb, y_train, validation_data=(x_val_rgb, y_valid), batch_size=256, epochs=50,
          callbacks=[tb_callback])

Train on 112800 samples, validate on 14100 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
  2048/112800 [..............................] - ETA: 3:31:27 - loss: 1.7718

KeyboardInterrupt: 